In [5]:
import pandas as pd

In [6]:
import os
print(os.getcwd())

/drive/test


In [7]:
csv=pd.read_csv('/drive/test/transactions.csv')

In [8]:
csv

,Transaction ID,Amount,Date
0,101,500,01-02-2024
1,102,700,02-02-2024
2,103,900,03-02-2024


In [9]:
csv1=pd.read_csv('/drive/test/transactions_1.csv')

In [10]:
csv1

,Transaction ID,Amount,Date
0,101,500,01-02-2024
1,102,800,02-02-2024
2,104,600,04-02-2024


In [11]:
import pandas as pd

def reconciliation_report(file1, file2):
    """Compares two transaction files and generates a reconciliation report."""
    try:
        df1, df2 = (pd.read_csv(file).set_index('Transaction ID') for file in [file1, file2])
        
        # Merges the datasets to find missing and mismatched transactions
        merged_df = pd.merge(df1, df2, on='Transaction ID', how='outer', suffixes=('_File1', '_File2'), indicator=True)
        
        missing_in_file2 = merged_df.query("_merge == 'left_only'").drop(columns=['_merge']).assign(Status='Missing in File 2')
        missing_in_file1 = merged_df.query("_merge == 'right_only'").drop(columns=['_merge']).assign(Status='Missing in File 1')
        
        mismatched_df = pd.DataFrame([
            {'Transaction ID': tid, 'File 1 Data': df1.loc[tid].to_dict(), 'File 2 Data': df2.loc[tid].to_dict(), 'Status': 'Mismatch'}
            for tid in df1.index.intersection(df2.index) if not df1.loc[tid].equals(df2.loc[tid])
        ]).set_index('Transaction ID') if not df1.empty and not df2.empty else pd.DataFrame()

        report = pd.concat([missing_in_file2, missing_in_file1, mismatched_df], sort=False)
        return report if not report.empty else "No differences found."

    except FileNotFoundError as e:
        return f"Error: {e}"
    except Exception as e:
        return f"An error occurred: {e}"

file1_path, file2_path = 'transactions.csv', 'transactions_1.csv'

print("Reconciliation Report:")
print(reconciliation_report(file1_path, file2_path))


Reconciliation Report:
                Amount_File1  Date_File1  Amount_File2  Date_File2  \
Transaction ID                                                       
103                    900.0  03-02-2024           NaN         NaN   
104                      NaN         NaN         600.0  04-02-2024   
102                      NaN         NaN           NaN         NaN   

                           Status                            File 1 Data  \
Transaction ID                                                             
103             Missing in File 2                                    NaN   
104             Missing in File 1                                    NaN   
102                      Mismatch  {'Amount': 700, 'Date': '02-02-2024'}   

                                          File 2 Data  
Transaction ID                                         
103                                               NaN  
104                                               NaN  
102             {'Amo

In [12]:
import pandas as pd

def reconcile_transactions(file_path_a, file_path_b):
    """
    Compare two Excel files based on 'Transaction ID' and identify discrepancies.
    Returns:
    tuple: (missing_in_a, missing_in_b, mismatched_amounts)
    """
    df_a = pd.read_csv(file_path_a)
    df_b = pd.read_csv(file_path_b)

    merged_df = df_a.merge(df_b, on="Transaction ID", how="outer", suffixes=('_A', '_B'))
    
    missing_in_a = merged_df[merged_df["Amount_A"].isna()].drop(columns=["Amount_A", "Date_A"])
    missing_in_b = merged_df[merged_df["Amount_B"].isna()].drop(columns=["Amount_B", "Date_B"])
    
    mismatched_amounts = merged_df.dropna(subset=["Amount_A", "Amount_B"])
    mismatched_amounts = mismatched_amounts[mismatched_amounts["Amount_A"] != mismatched_amounts["Amount_B"]]
    
    return missing_in_a, missing_in_b, mismatched_amounts

file_path_a = ('/drive/test/transactions.csv')
file_path_b = ('/drive/test/transactions_1.csv')

missing_in_a, missing_in_b, mismatched_amounts = reconcile_transactions(file_path_a, file_path_b)

print("Missing in A:")
print(missing_in_a)

print("\nMissing in B:")
print(missing_in_b)

print("\nMismatched Amounts:")
print(mismatched_amounts)


Missing in A:
   Transaction ID  Amount_B      Date_B
3             104     600.0  04-02-2024

Missing in B:
   Transaction ID  Amount_A      Date_A
2             103     900.0  03-02-2024

Mismatched Amounts:
   Transaction ID  Amount_A      Date_A  Amount_B      Date_B
1             102     700.0  02-02-2024     800.0  02-02-2024
